## Problem Statement

In this exercise we will scrape twitter data through twitter's API and do a tf-idf analysis on that. We will need OAuth authentication. 

In [1]:
import oauth2 as oauth
import urllib2 as urllib
import json

We will now need twitter api access. The following steps as available online will help you set up your twitter account and access the live 1% stream.

1. Create a twitter account if you do not already have one.
2. Go to https://dev.twitter.com/apps and log in with your twitter credentials.
3. Click "Create New App"
4. Fill out the form and agree to the terms. Put in a dummy website if you don't have one you want to use.
5. On the next page, click the "API Keys" tab along the top, then scroll all the way down until you see the section "Your Access Token"
6. Click the button "Create My Access Token". You can Read more about Oauth authorization online. 

Save the details of api_key, api_secret, access_token_key, access_token_secret in your vaule directory and load it in the notebook as shown in yelpSample notebook.

In [2]:
import sys
sys.path.append('/Users/cjstev/Documents/DSE/c1steven/DSE200/DSE200-notebooks/day_5_mining_the_Social_web/')
import twitterKeys
api_key,api_secret,access_token_key,access_token_secret = twitterKeys.getkeys()

_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"

http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

Below is a twitter request method which will use the above user logins to sign, and open a twitter stream request

In [3]:
def getTwitterStream(url, method, parameters):
    req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=method,
                                             http_url=url, 
                                             parameters=parameters)

    req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

    headers = req.to_header()

    if http_method == "POST":
        encoded_post_data = req.to_postdata()
    else:
        encoded_post_data = None
        url = req.to_url()

    opener = urllib.OpenerDirector()
    opener.add_handler(http_handler)
    opener.add_handler(https_handler)

    response = opener.open(url, encoded_post_data)

    return response

We can use the above function to request a response as follows

In [4]:
#Now we will test the above function for a sample data provided by twitter stream here -  
url = "https://stream.twitter.com/1/statuses/sample.json"

In [5]:
parameters = []
response = getTwitterStream(url, "GET", parameters)
#response.readline()

Write a function which will take a url and return the top 10 lines returned by the twitter stream

** Note ** The response returned needs to be intelligently parsed to get the text data which correspond to actual tweets. This part can be done in a number of ways and you are encouraged to try different approaches to parse the response data.

In [6]:
def fetchData(url,numreturns=10,searchquery='N'):
    if searchquery=='N':
        top10 = []
        response = getTwitterStream(url, "GET", parameters)
        while len(top10)<numreturns:

            testtweet = response.readline()
            json_acceptable_string = testtweet.replace("'", "\"")

            try:
                tweet = json.loads(json_acceptable_string)
                text = tweet['text']
                top10.append( tweet['text'])
            except:
                continue
        return top10
    else:
        response = getTwitterStream(url, "GET", parameters)
        testtweet = response.readline()
        json_acceptable_string = testtweet.replace("'", "\"")
        tweet = json.loads(testtweet)
        topx=[]
        for i in range(numreturns):
            topx.append(tweet['statuses'][i]['text'])
        return topx

In [7]:
tweetlist = fetchData(url)
for i in range(len(tweetlist)):
    print i+1, tweetlist[i]

1 RT @madamlesexy: PEKİ AŞKIM BANA BİRAZ GEÇMİŞİNDEN BAHSETSENE, MESELA ESKİ SEVGİLİLERİNLE NELER YAŞADIN https://t.co/iRBC2H7fP1
2 RT @neews_sports: الشوط الاول

روابط بث مباشر
مباراة #الاهلي_الشباب
يوتيوب
https://t.co/iAb7zz69XD
للجوال
https://t.co/Ay9xSYAut7

رابط مبا…
3 RT @slimG_07: 16. a Baltimore classic https://t.co/lOzk3PqWnu
4 RT @f14_foo: ❗️❗️عاجل ❗️❗️

نشتري القروض العقارية التي صدر لها الموافقة من الصندوق العقاري من جميع المناطق 

أبونايف : 0553880031
5 RT @DepressedDarth: Sticks and stones may break my bones but lightsabers will chop my limbs off
6 RT @BookNerdParadis: Discover authors new to you as our guest authors read excerpts at https://t.co/5IBGJyDLpR #books #author #IAN1 #CR4U h…
7 RT @qxo1YmpOrKcnXxE: اللهم إني أسألك علماً نافعاً ورزقاً طيباً وعملاً متقبل
♻️ https://t.co/wK5C8FUH9W
8 RT @kenan_kiran: Yavuz Sultan Selim Köprüsü hayırlı olsun.. https://t.co/UcoT0UBeOe
9 RT GreenBison1867 "RT fahreal_tho: #Howard_RYS16 amartin126: GreenBison1867 vote again. #Howard_R

In [8]:
#We can also request twitter stream data for specific search parameters as follows
#url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query

Call the fetchData function to fetch latest live stream data for following search queries and output the first 5 lines

1. "UCSD"
2. "Donald Trump"
3. "Syria"

In [9]:
search_query = "UCSD"
url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query
tweetlist = fetchData(url,5,'Y')
for i in range(len(tweetlist)):
    print i+1, tweetlist[i]

1 3R 2H 0E (WLDC) | UCSD 3, WLDC 6 | End 3rd Recap | https://t.co/BrB1qki7ER
2 0R 1H 0E (UCSD) | UCSD 3, WLDC 3 | Mid 3rd Recap | https://t.co/BrB1qki7ER
3 @Btrillaaa In LA, Loyola Marymount/USC, uc riverside, then San Diego state and ucsd
4 3R 4H 0E (WLDC) | UCSD 3, WLDC 3 | End 2nd Recap | https://t.co/BrB1qki7ER
5 RT @Crimbleit: Change Safari’s Default Search Engine in OS X 10 https://t.co/dR1hFmq6tM via #itsupport #college #university #mac #UCLA #UCS…


In [10]:
search_query = "Donald Trump"
url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query
tweetlist = fetchData(url,5,'Y')
for i in range(len(tweetlist)):
    print i+1, tweetlist[i]

1 RT @YourFavoriteZoe: This happened at a presidential candidate rally... 

Yup, Donald Trump.  https://t.co/ct702j1UYQ
2 Watch 'SNL' Mock Racist Donald Trump Supporters in Campaign Ad https://t.co/Y3dibd0yqa
3 On ABC's This Week , Cokie Roberts Says GOP Has Only Itself To Blame For Donald Trump's Rise https://t.co/yHuYFYFCKn
4 LOL Chris Wallace vs. Donald Trump https://t.co/Y5lu1wagSA
5 RT @quitfeenin: 24. At Donald Trump Rally's: Harmless protestors got their signs ripped and thrown out. #DumpTrump https://t.co/KJzQj6K5ZZ


In [11]:
search_query = "Syria"
url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query
tweetlist = fetchData(url,5,'Y')
for i in range(len(tweetlist)):
    print i+1, tweetlist[i]

1 RT @yo7521: مباراه الشباب والاهلي HD
يوتيوب  https://t.co/5yWx8lrQ44   
جوال    https://t.co/5yWx8lrQ44   

#الشباب_الاهلي
https://t.co/KKS…
2 RT @yo7521: مباراه الشباب والاهلي HD
يوتيوب  https://t.co/5yWx8lrQ44   
جوال    https://t.co/5yWx8lrQ44   

#الشباب_الاهلي
https://t.co/KKS…
3 RT @yo7521: مباراه الشباب والاهلي HD
يوتيوب  https://t.co/5yWx8lrQ44   
جوال    https://t.co/5yWx8lrQ44   

#الشباب_الاهلي
https://t.co/KKS…
4 RT @yo7521: مباراه الشباب والاهلي HD
يوتيوب  https://t.co/5yWx8lrQ44   
جوال    https://t.co/5yWx8lrQ44   

#الشباب_الاهلي
https://t.co/KKS…
5 RT @yo7521: مباراه الشباب والاهلي HD
يوتيوب  https://t.co/5yWx8lrQ44   
جوال    https://t.co/5yWx8lrQ44   

#الشباب_الاهلي
https://t.co/KKS…


### TF-IDF###

tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is among the most regularly used statistical tool for word cloud analysis. You can read more about it online (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

We base our analysis on the following

1. The weight of a term that occurs in a document is simply proportional to the term frequency
2. The specificity of a term can be quantified as an inverse function of the number of documents in which it occurs

For this question we will perform tf-idf analysis o the stream data we retrieve for a given search parameter. Perform the steps below

1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt
2. load the saved file and output the count of occurrences for each term. This will be your term frequency
3. Calculate the inverse document frequency for each of the term in the output above.
4. Divide the term frequency for each of the term by corresponding inverse document frequency.
5. Sort the terms in the descending order based on their term freq/inverse document freq scores 
6. Print the top 10 terms.

In [12]:
search_query = "syria"   ## chooose query
url= "https://api.twitter.com/1.1/search/tweets.json?lang=en&count=100&q="+search_query
response = getTwitterStream(url, "GET", parameters)
testtweet = response.read()
tweet = json.loads(testtweet)  ## convert json
topx=[]
for i in range(100):
    topx.append(tweet['statuses'][i]['text']) ## pull tweets into list


In [13]:
topx = [tweet.replace('\n',' ') for tweet in topx] # put each tweet on one line

In [14]:
outfile = open('twitterStream.txt', 'w')
outfile.write(u"\n".join(topx).encode('utf-8').strip()) ## write to file

In [15]:
infile = open('twitterStream.txt', 'r')
text = infile.read()  #read back file

In [16]:
import string
exclude = set(string.punctuation)  
text = ''.join(ch for ch in text if ch not in exclude)  #drop punctuation
text = text.lower()  #convert to lowercase
import re
word_list=re.split('\s+',text)  ## split words into wordlist

In [17]:
## create column for number of lines the word appears on
numlines = []
for word in word_list:
    linecount = [word in line for line in text.splitlines()]
    numlines.append(sum(linecount))

In [18]:
## create column for number of times the word appears overall
numtimes = []
for word in word_list:
    linecount = [word in elem for elem in word_list]
    numtimes.append(sum(linecount))

In [19]:
## throw it all into a pandas DF
import pandas as pd
df2 = pd.DataFrame({"Words":word_list,"numlines":numlines,"numtimes":numtimes})
df3 = df2.drop_duplicates().sort_values('numlines',ascending=False)  # the way i created this will have duplicates, so drop them

In [20]:
import numpy as np
df3['idf'] = np.log(100/df3['numlines'])  ## create idf col
df3['tf'] = np.log(1+df3['numtimes'])  ## create tf col
df3['score'] = df3['tf'] * df3['idf']  ## score them

In [21]:
df3.sort_values('score',ascending=False,inplace=True)
df3.head(10)

,Words,numlines,numtimes,idf,tf,score
316,our,6,10,2.813411,2.397895,6.746264
33,kurdish,10,17,2.302585,2.890372,6.655327
1139,million,1,3,4.605170,1.386294,6.384121
1200,year,3,5,3.506558,1.791759,6.282908
743,hillary,3,5,3.506558,1.791759,6.282908
144,my,5,7,2.995732,2.079442,6.229450
1455,ted,7,9,2.659260,2.302585,6.123173
176,iran,13,17,2.040221,2.890372,5.896997
896,day,6,7,2.813411,2.079442,5.850323
294,russia,7,8,2.659260,2.197225,5.842992
